In [ ]:
import os
import ast
import math
import json
import torch
import openai
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import Counter
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from langchain.embeddings import OpenAIEmbeddings
from transformers import BertTokenizer, BertModel
from get_embedding import getEmbeddingDict


In [117]:
import pandas as pd
book_data = pd.read_excel("Book.xlsx")
book_df = book_data[['tags_emotion', 'label']]
book_test_df = pd.read_excel("Book_test.xlsx")


In [118]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the OneHotEncoder and Tokenizer
onehot_encoder = OneHotEncoder(sparse=False)
tokenizer = Tokenizer(num_words=5000)

# One-hot encode the labels
labels = onehot_encoder.fit_transform(book_df['label'].values.reshape(-1, 1))

# Fit the tokenizer on the text data
tokenizer.fit_on_texts(book_df['tags_emotion'])

# Convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(book_df['tags_emotion'])
sequences_test = tokenizer.texts_to_sequences(book_test_df['tags_emotion'])

# Pad the sequences so that they are all the same length
max_sequence_length = max(max(len(seq) for seq in sequences), max(len(seq) for seq in sequences_test))
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_sequence_length)

# Split the "Book" data into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

(X_train.shape, y_train.shape), (X_val.shape, y_val.shape), (padded_sequences_test.shape)


/opt/homebrew/Caskroom/miniforge/base/envs/cq1/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(((91, 1), (91, 10)), ((23, 1), (23, 10)), (192, 1))

In [119]:
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import torch

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
encoded_labels = label_encoder.fit_transform(book_df['label'])

# Convert labels to one-hot encoding
labels_one_hot = torch.nn.functional.one_hot(torch.tensor(encoded_labels))

# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(book_df['tags_emotion'])
sequences = tokenizer.texts_to_sequences(book_df['tags_emotion'])
sequences_test = tokenizer.texts_to_sequences(book_test_df['tags_emotion'])

# Pad sequences
max_sequence_length = max(max(len(seq) for seq in sequences), max(len(seq) for seq in sequences_test))
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_sequence_length)

# Convert to PyTorch tensors
X = torch.tensor(padded_sequences, dtype=torch.long)
X_test = torch.tensor(padded_sequences_test, dtype=torch.long)
y = labels_one_hot.float()

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create TensorDatasets for training and validation
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)

# DataLoader parameters
batch_size = 32

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size)
val_loader = DataLoader(val_data, batch_size=batch_size)

(X_train.shape, y_train.shape), (X_val.shape, y_val.shape), (X_test.shape)

((torch.Size([91, 1]), torch.Size([91, 10])),
 (torch.Size([23, 1]), torch.Size([23, 10])),
 torch.Size([192, 1]))

In [120]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder

# Initialize the TF-IDF Vectorizer and LabelEncoder
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
label_encoder = LabelEncoder()

# Encode the labels to integers
encoded_labels = label_encoder.fit_transform(book_df['label'])

# Create a TF-IDF representation of the text data
tfidf_matrix = tfidf_vectorizer.fit_transform(book_df['tags_emotion'])

# Split the data into a training set and a validation set
X_train, X_val, y_train, y_val = train_test_split(tfidf_matrix, encoded_labels, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf_classifier.fit(X_train, y_train)

# Evaluate the model on the validation set
val_score = rf_classifier.score(X_val, y_val)

val_score


0.43478260869565216

In [121]:
# Transform the "Book_test" data using the fitted TF-IDF vectorizer
tfidf_matrix_test = tfidf_vectorizer.transform(book_test_df['tags_emotion'])

# Predict the labels for the "Book_test" data
test_predictions = rf_classifier.predict(tfidf_matrix_test)

# Convert the integer predictions back to the original labels
test_predicted_labels = label_encoder.inverse_transform(test_predictions)

# Add the predicted labels to the test dataframe
book_test_df['predicted_label'] = test_predicted_labels

# Show the first few rows of the dataframe with predictions
book_test_df.head()


,tags_emotion,predicted_label
0,压迫,5
1,压抑,5
2,残忍,5
3,残暴,5
4,兴奋,2


In [115]:
book_test_df.sort_values('predicted_label').to_excel('Book_draft.xlsx')

In [107]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load your data here
book_df = pd.read_excel('Book.xlsx')
book_test_df = pd.read_excel('Book_test.xlsx')

# Assume book_df has columns 'tags_emotion' and 'label'
texts = book_df['tags_emotion'].values
labels = book_df['label'].values

# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences to the same length
max_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_length)

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
y = tf.keras.utils.to_categorical(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=max_length))
model.add(SimpleRNN(units=64, return_sequences=False))
model.add(Dense(units=y.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))
model.save('model.h5')


Epoch 1/30
3/3 [==============================] - 1s 55ms/step - loss: 2.2940 - accuracy: 0.2088 - val_loss: 2.2853 - val_accuracy: 0.2609
Epoch 2/30
3/3 [==============================] - 0s 8ms/step - loss: 2.2663 - accuracy: 0.3736 - val_loss: 2.2759 - val_accuracy: 0.2609
Epoch 3/30
3/3 [==============================] - 0s 8ms/step - loss: 2.2414 - accuracy: 0.4396 - val_loss: 2.2659 - val_accuracy: 0.3043
Epoch 4/30
3/3 [==============================] - 0s 9ms/step - loss: 2.2137 - accuracy: 0.5275 - val_loss: 2.2557 - val_accuracy: 0.3043
Epoch 5/30
3/3 [==============================] - 0s 9ms/step - loss: 2.1865 - accuracy: 0.5385 - val_loss: 2.2450 - val_accuracy: 0.3478
Epoch 6/30
3/3 [==============================] - 0s 9ms/step - loss: 2.1552 - accuracy: 0.5385 - val_loss: 2.2340 - val_accuracy: 0.3478
Epoch 7/30
3/3 [==============================] - 0s 9ms/step - loss: 2.1226 - accuracy: 0.5604 - val_loss: 2.2223 - val_accuracy: 0.3478
Epoch 8/30
3/3 [=================

In [108]:
# Tokenize and pad the 'Book_test' text data
test_texts = book_test_df['tags_emotion'].values
test_sequences = tokenizer.texts_to_sequences(test_texts)
X_test = pad_sequences(test_sequences, maxlen=max_length)  # max_length used during training

# Predict with the model
predictions = model.predict(X_test)

# If your model outputs one-hot encoded predictions, you'll need to convert these to label encodings
predicted_labels = predictions.argmax(axis=-1)

# Convert the integer predictions back to the original labels using the label encoder
predicted_labels = label_encoder.inverse_transform(predicted_labels)

# Add the predicted labels to the test dataframe if needed
book_test_df['predicted_label'] = predicted_labels


6/6 [==============================] - 0s 619us/step


In [122]:
diction = {1:'搞笑组', 2:'情绪组', 3:'害怕组', 4:'正能量组', 5:'负能量组', 6:'烂片组', 7:'题材', 8:'主角路线', 9:'励志组', 10:'荒诞组', 11:'思考组', 12:'经典组'}

df = book_test_df.sort_values('predicted_label')
df['predicted_label'] = df['predicted_label'].map(diction)
df = df[['tags_emotion', 'predicted_label']]
df.to_excel('Book_output.xlsx')

In [123]:
book_test_df.sort_values('predicted_label').head(50)

,tags_emotion,predicted_label
128,幽默,1
29,幸福,2
45,欢乐,2
118,轻松,2
44,欢笑,2
80,温馨,2
28,感人,2
4,兴奋,2
97,喜悦,2
188,开心,2


In [71]:
print(len(book_test_df[book_test_df['predicted_label']==12]))
book_test_df[book_test_df['predicted_label']==12]


0


,tags_emotion,predicted_label


In [72]:
book_test = pd.read_excel('Book_draft.xlsx')
print(len(book_test[book_test['predicted_label']==12]))
book_test[book_test['predicted_label']==12]

0


,tags_emotion,predicted_label


In [51]:
for i in b:
    if i not in a:
        print(i)

抑郁
友爱
欢乐
感动
腐败
无助
自省
复仇


In [64]:
book_test[book_test['tags_emotion'] == '奋斗']

,tags_emotion,predicted_label
26,奋斗,4


In [85]:
tuple_pca_embedding_dict = {k: tuple(v) for k, v in diction.items()}
inverse_pca_embedding_dict = {v: k for k, v in tuple_pca_embedding_dict.items()}
book_test_df['predicted_label'] = book_test_df['predicted_label'].apply(lambda x: inverse_pca_embedding_dict[tuple(x)] if tuple(x) in inverse_pca_embedding_dict else None)
book_test_df = book_test_df[['tags_emotion', 'predicted_label']]

In [97]:
diction = {1:'搞笑组', 2:'情绪组', 3:'害怕组', 4:'正能量组', 5:'负能量组', 6:'烂片组', 7:'题材', 8:'主角路线', 9:'励志组', 10:'荒诞组', 11:'思考组', 12:'经典组'}

df = book_test_df.sort_values('predicted_label')
df['predicted_label'] = df['predicted_label'].map(diction)
df = df[['tags_emotion', 'predicted_label']]
df

,tags_emotion,predicted_label
128,幽默,搞笑组
80,温馨,情绪组
188,开心,情绪组
44,欢笑,情绪组
4,兴奋,情绪组
...,...,...
66,震撼,荒诞组
12,疑惑,思考组
83,自省,思考组
101,遗憾,思考组


In [98]:
df.to_excel('Book_output.xlsx')

In [93]:
# Train the model, capturing the history
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

# The history object now contains the records of loss and accuracy for each epoch
print(history.history)

import matplotlib.pyplot as plt

# Plot training & validation accuracy values
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# # plt.legend(['Train'], loc='upper left')
# # plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
plt.show()



Epoch 1/50


4/4 [==============================] - 0s 12ms/step - loss: 0.0245 - accuracy: 0.9832 - val_loss: 2.2969 - val_accuracy: 0.2333
Epoch 2/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0247 - accuracy: 0.9748 - val_loss: 2.2967 - val_accuracy: 0.2333
Epoch 3/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0246 - accuracy: 0.9748 - val_loss: 2.2968 - val_accuracy: 0.2333
Epoch 4/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0250 - accuracy: 0.9832 - val_loss: 2.2972 - val_accuracy: 0.2333
Epoch 5/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0242 - accuracy: 0.9832 - val_loss: 2.2971 - val_accuracy: 0.2333
Epoch 6/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0247 - accuracy: 0.9748 - val_loss: 2.2974 - val_accuracy: 0.2333
Epoch 7/50
4/4 [==============================] - 0s 7ms/step - loss: 0.0240 - accuracy: 0.9832 - val_loss: 2.2974 - val_accuracy: 0.2333
Epoch 8/50
4/4 [============================

TypeError: unsupported operand type(s) for -: 'list' and 'list'